In [1]:
import pandas as pd
import numpy as np
%matplotlib
from matplotlib import pyplot as plt

Using matplotlib backend: MacOSX


In [185]:
# download data from 
# http://governacio.gencat.cat/ca/pgov_ambits_d_actuacio/pgov_eleccions/pgov_dades_electorals/

### INTRODUCTION

Regional elections in Catalonia on 27 September 2015  were not ordinary ones. They were a referendum in disguise. A referendum on the independence of Catalonia from the Spanish state. IDESCAT is the national institute of statistics of Catalonia.

#### IMPORT AND PROCESS ELECTORAL DATA

In [14]:
# import codes, pct votes, Candid
df_pct_vote_info = pd.read_csv('A20151_MU/FilesVots_A20151_MU_ca_ES.csv',sep =';',encoding='latin-1') 
# turn pct data from strings into integers ( first need to change comma into decimal point)
df_pct_vote_info["% vàlids"] = pd.to_numeric(df_pct_vote_info['% vàlids'].str.replace(",",".")) 
# make pivot table to rearrange info
df_pct_vote_info = df_pct_vote_info.pivot_table(index = ["Nom Municipi", "Codi Municipi", "Codi Província"], columns = "Candidatures", values = "% vàlids")
df_pct_vote_info = df_pct_vote_info.filter( regex = r"(Junt|Ciuta|Sociali|Partit Pop|es Pot|Unitat|Democr|Codi Mun)" )
# rename columns (change name Candidatures to make it simpler)
df_pct_vote_info.columns = ["CUP_pct","CSQP_pct","Cs_pct","JxSi_pct","PP_pct","PSC_pct","Unio_pct"]
# reset index
df_pct_vote_info = df_pct_vote_info.reset_index().set_index('Nom Municipi')
## Add columns
df_pct_vote_info['Unionisme_pct'] = df_pct_vote_info['Cs_pct'] + df_pct_vote_info['PSC_pct'] + df_pct_vote_info['PP_pct']
df_pct_vote_info['Independ_pct'] = df_pct_vote_info['JxSi_pct'] + df_pct_vote_info['CUP_pct']

In [94]:
df_codes = pd.read_html('https://www.idescat.cat/codis/?id=50&n=9')[0]
df_codes.columns = ['Codi', 'Nom', 'Codi comarca', 'Nom comarca']
df_codes = df_codes.set_index('Nom')
df_codes.to_csv('Municip_codes_from_IDESCAT.csv')

#### EXAMPLES TO UNDERSTAND DF MERGE, JOIN

In [129]:
d1 = pd.DataFrame({'a':[1,2], 'b':[3,4]})
d2 = pd.DataFrame({'c':[11,22],'d':[33,44]})

In [130]:
d1.join(d2)

,a,b,c,d
0,1,3,11,33
1,2,4,22,44


In [152]:
dummy_1 = pd.DataFrame({'codis':['aa','bb','cc'], 'val1':[2,5,7]}).set_index('codis')
dummy_2 = pd.DataFrame({'codis':['aa','bb','dd'], 'val2':[21,52,74]}).set_index('codis')

In [155]:
dummy_2.join(dummy_1)

,val2,val1
codis,,
aa,21,2
bb,52,5
dd,74,NaN


In [156]:
dummy_1.join(dummy_2)

,val1,val2
codis,,
aa,2,21
bb,5,52
cc,7,NaN


In [163]:
pd.merge(dummy_1, dummy_2, left_index=True, right_index=True, how='outer')

,val1,val2
codis,,
aa,2,21
bb,5,52
cc,7,NaN
dd,NaN,74


In [95]:
pd.merge(df_pct_vote_info, df_codes, left_index=True, right_index=True, how='outer')

In [96]:
# url = 'http://www.idescat.cat/emex/?id=080327'
# id_scrap = requests.get(url, headers=headers)
# soup = BeautifulSoup(id_scrap.text, "lxml")
# table = soup.findAll("table", id ='t90')
# xx = pd.read_html(str(table), thousands='.')[0]

In [97]:
import requests
from bs4 import BeautifulSoup
import json

headers = {'User-Agent':
           "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/601.4.4 (KHTML, like Gecko) Version/9.0.3 Safari/601.4.4"}

list_ids = ['t68','t56','t84','t90','t5']
labels = ['Pop_struct', 'Unemployment', 'Educ_level', "Català", 'Income_level']

df_codes = pd.read_csv('Municip_codes_from_IDESCAT.csv')
codes, names = df_codes['Codi'].astype(str).str.zfill(6).values, df_codes['Nom'].values

all_munic_info = defaultdict(dict)

for code, name in zip(codes, names):
    print(name)
    url = 'http://www.idescat.cat/emex/?id=' + code
    req_resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(req_resp.text, "lxml")
    for idx, label in zip(list_ids, labels):
        try:
            table = soup.findAll("table", id =idx)
            values = pd.read_html(str(table), thousands='.')[0][name].str.replace(",", ".").values[:-1]
            all_munic_info[name][label] = [float(x) if '.' in x 
                                           else None if ':' in x  
                                           else int(x) 
                                           for x in values]
        except:
            all_munic_info[name][label] = []
#save data            
with open('ALL_MUNICIPAL_DATA.json', 'w') as f:
    json.dump(all_munic_info, f)         

In [4]:
with open('ALL_MUNICIPAL_DATA.json') as data_file:    
    data = json.load(data_file)

#### Make dataframes from json data

In [139]:
pop_stats, ed_level_stats, cat_stats = [], [], []
names_municips = []
for idx, (key, val) in enumerate(data.items()):
    pop_stats.append(val['Pop_struct'])
    ed_level_stats.append(val['Educ_level'])
    cat_stats.append(val['Català'])
    names_municips.append(key)
pop_stats = pd.DataFrame(pop_stats, columns =['Cat', 'Spa', 'Abroad', 'Tot'])
ed_level_stats = pd.DataFrame(ed_level_stats, columns =['ST', '1erGrau', '2nGrau','Univ','Tot_Niv_Ed'])
cat_stats = pd.DataFrame(cat_stats, columns = ['understands', 'speaks', 'reads', 'writes', 'no_underst', 'Tot_cat'])
names_municips = pd.DataFrame(names_municips, columns=['Municipi'])

In [140]:
# add new columns to data dfs
pop_stats['pct_foreign'] = 100 * pop_stats['Abroad'] / pop_stats['Tot']
pop_stats['pct_spa'] = 100 * pop_stats['Spa'] / pop_stats['Tot']
ed_level_stats['pct_Univ'] = 100 * ed_level_stats['Univ']/ed_level_stats['Tot_Niv_Ed']
cat_stats['pct_cat_speakers'] = 100 * cat_stats['speaks'] / cat_stats['Tot_cat']

all_stats = pop_stats.join(ed_level_stats).join(cat_stats).join(names_municips)
all_stats = all_stats.set_index('Municipi')

# all_stats = pd.merge(pop_stats, ed_level_stats)
# all_stats = pd.merge(all_stats, cat_stats).set_index('Municipi')

In [141]:
DF_ALL_MUNIC_DATA = df_pct_vote_info.join(all_stats)

In [142]:
DF_ALL_MUNIC_DATA.head()

,Codi Municipi,Codi Província,CUP_pct,CSQP_pct,Cs_pct,JxSi_pct,PP_pct,PSC_pct,Unio_pct,Unionisme_pct,...,Univ,Tot_Niv_Ed,pct_Univ,understands,speaks,reads,writes,no_underst,Tot_cat,pct_cat_speakers
Nom Municipi,,,,,,,,,,,,,,,,,,,,,
Abella de la Conca,25001,25,16.82,2.80,8.41,53.27,4.67,10.28,2.80,23.36,...,NaN,NaN,NaN,169,164,161,107,0,169,97.041420
Abrera,8001,8,6.43,13.31,25.14,23.92,8.79,18.10,1.86,52.03,...,NaN,9465,NaN,10796,7703,8748,6099,556,11351,67.861862
Agramunt,25003,25,5.21,3.56,8.64,66.48,6.34,5.52,2.98,20.50,...,NaN,4601,NaN,5344,4472,4582,3692,NaN,5416,82.570162
Aguilar de Segarra,8002,8,10.98,3.05,2.44,73.78,2.44,0.00,6.10,4.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Agullana,17001,17,16.77,3.35,6.50,62.47,3.14,5.45,1.89,15.09,...,NaN,NaN,NaN,773,673,730,522,NaN,819,82.173382


In [143]:
plt.scatter(DF_ALL_MUNIC_DATA['pct_spa'], DF_ALL_MUNIC_DATA['Unionisme_pct'], color='blue', edgecolors='black')
plt.scatter(DF_ALL_MUNIC_DATA['pct_spa'], DF_ALL_MUNIC_DATA['Independ_pct'], color='yellow', edgecolors='b')
plt.xlim(0, 35)
plt.ylim(0, 100)

(0, 100)

In [144]:
#plt.scatter(DF_ALL_MUNIC_DATA['pct_cat_speakers'], DF_ALL_MUNIC_DATA['Unionisme_pct'], s= DF_ALL_MUNIC_DATA['Tot']/100)
plt.scatter(DF_ALL_MUNIC_DATA['pct_cat_speakers'], DF_ALL_MUNIC_DATA['Independ_pct'],
            edgecolors='b',
            s= DF_ALL_MUNIC_DATA['Tot']/100, 
            color='y',
            alpha=0.6)

In [145]:
plt.scatter(DF_ALL_MUNIC_DATA['pct_cat_speakers'], DF_ALL_MUNIC_DATA['pct_spa'] + DF_ALL_MUNIC_DATA['pct_foreign'])

In [146]:
plt.scatter( DF_ALL_MUNIC_DATA['pct_Univ'], DF_ALL_MUNIC_DATA['Independ_pct'], s= DF_ALL_MUNIC_DATA['Tot']/100, alpha=0.7)

In [147]:
DF_ALL_MUNIC_DATA.keys()

Index(['Codi Municipi', 'Codi Província', 'CUP_pct', 'CSQP_pct', 'Cs_pct',
       'JxSi_pct', 'PP_pct', 'PSC_pct', 'Unio_pct', 'Unionisme_pct',
       'Independ_pct', 'Cat', 'Spa', 'Abroad', 'Tot', 'pct_foreign', 'pct_spa',
       'ST', '1erGrau', '2nGrau', 'Univ', 'Tot_Niv_Ed', 'pct_Univ',
       'understands', 'speaks', 'reads', 'writes', 'no_underst', 'Tot_cat',
       'pct_cat_speakers'],
      dtype='object')

In [148]:
####STATSMODELS REGRESSION ####################
## before symbolic regression get rid of blank spaces in column names
DF_ALL_MUNIC_DATA.columns = DF_ALL_MUNIC_DATA.columns.str.replace(" ","_")
#import libraries
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Fit regression model()
results = smf.ols("Independ_pct ~ pct_foreign + pct_Univ ", data = DF_ALL_MUNIC_DATA).fit()
## Inspect the results
print ( results.summary())

                            OLS Regression Results                            
Dep. Variable:           Independ_pct   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     13.89
Date:                Tue, 29 Aug 2017   Prob (F-statistic):           2.93e-06
Time:                        09:55:42   Log-Likelihood:                -616.10
No. Observations:                 152   AIC:                             1238.
Df Residuals:                     149   BIC:                             1247.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      36.6028      4.302      8.508      